# 기초 작업

### 모듈 불러오기

In [1]:
import selenium
from selenium import webdriver as wd
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import repeat
import sqlite3
from pymongo import MongoClient

### 크롬드라이버가 url 접속하도록 하기

In [2]:
# 창 실행 옵션x
options = wd.ChromeOptions()
options.add_argument("headless") 

# 크롬드라이버 열기
driver = wd.Chrome('chromedriver', chrome_options=options) # 크롬드라이버 경로

# 드라이버가 해당 url 접속
url = 'https://www.melon.com/chart/index.htm' # 멜론차트 페이지
driver.get(url)

<ipython-input-2-264a2b05d9a8>:6: DeprecationWarning: use options instead of chrome_options
  driver = wd.Chrome('chromedriver', chrome_options=options) # 크롬드라이버 경로


# 함수 만들기

### 클릭 함수 만들기

In [3]:
# 연대선택 era_num=1 : 2020년대, era_num=2 : 2010년대
def click_era(era_num) :    
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li['+str(era_num)+']/span/label').click()
    time.sleep(0.1)
    
# 연도선택 year_num : 최근 -> 과거
def click_year(year_num) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li['+str(year_num)+']/span/label').click()
    time.sleep(0.1)

# 월간선택 
def click_month(month_year) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li['+str(month_year)+']/span/label').click()
    time.sleep(0.1)

# 주간 선택
def click_week(week_num) :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li['+str(week_num)+']/span/label').click()
    time.sleep(0.1)

# 장르선택 ('장르 종합'으로 선택)
def click_genre() :
    driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
    time.sleep(0.1)
    
    
# 검색버튼 클릭
def click_search() :
    driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()
    time.sleep(0.1)



### 선택 개수 찾기 함수 만들기

In [4]:
# 연대선택 개수 찾기
def find_era_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth1.view > div.list_value > ul > li'))
    return length    
    
# 연도선택 개수 찾기
def find_year_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth2.view > div.list_value > ul > li'))
    return length

# 월간선택 개수 찾기
def find_month_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth3.view > div.list_value > ul > li'))
    return length

# 주간선택 개수 찾기
def find_week_length() :
    length = len(driver.find_elements_by_css_selector('div.box_chic.nth4.view > div.list_value > ul > li'))
    return length

### 크롤링 및 저장 함수 만들기

In [5]:
# 크롤링 & 저장하기
def crawling_and_save_to_csv(year_num, month_num, week_num) :
    
    # html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    #순위 가져오기
    song = [title.find('strong').get_text() for title in soup.find_all('div', attrs={'class': 'ellipsis rank01'})]
    ranks = []
    for i in range(len(song)):
        ranks.append(i+1)
    
    # 가수명 가져오기
    artist_name_list = []
    for artist_name in soup.find_all('span', attrs={'class':'checkEllipsis'}) :
        artist_name_list.append(artist_name.get_text())
    
    # 곡명 가져오기
    song_name_list = []
    for song_name in soup.find_all('div', attrs={'class': 'ellipsis rank01'}) :
        song_name_list.append(song_name.find('strong').get_text())
    
    # 곡 아이디 가져오기
    song_id_list=[]
    for song_id in soup.find_all('input',{'name':'input_check'}):
        song_id_list.append(song_id.get('value'))
        
    # 파일 제목 정하기    
    year = soup.find('input',{'id':'year_'+str(year_num)}).get('value')
    month = soup.find('input',{'id':'month_'+str(month_num)}).get('value')
    week = week_num
    date = str(year)+'-'+str(month)+'-'+str(week)
    
    
    
    # 데이터프레임 만들기
    df = pd.DataFrame({"rank" : ranks, "artist_name" : artist_name_list, "song_name" : song_name_list, "song_id" : song_id_list})
    
    # csv로 저장
    df.to_csv('csv/song_TOP100_list/'+date+'_TOP100.csv',encoding='utf-8-sig', index=False)
    
    # SQLite3 DB로 저장
    con = sqlite3.connect("db/song.db")
    df.to_sql(date+"_TOP100", con, if_exists="replace", index=False)
    con.close()
    
    # MongoDB에 저장
    client = MongoClient('localhost', 27017)
    db = client['song']
    col = db[date+"_TOP100"]
    for i in range(len(ranks)) :
        doc = {"rank" : ranks[i], "artist_name" : artist_name_list[i], "song_name" : song_name_list[i], "song_id" : song_id_list[i]}
        col.insert_one(doc)
    
    
    print(date+" ", end = "")
    
    

# 크롤링

In [6]:
# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()

# 주간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[1]/a').click()
time.sleep(0.1)

# 나머지 멈추는 시기 결정
stop_era = 2
stop_year = 2
stop_month = 6

# 나머지 자동 선택
song_id_list =[]
len_era = find_era_length()
for i in range(1,len_era+1) :   
    click_era(i)
        
    len_year = find_year_length()
    for j in range(1,len_year+1) :
        click_year(j)
                
        len_month = find_month_length()
        
        if i == stop_era and j == stop_year :
            for k in range(stop_month, len_month+1) :
                click_month(k)

                len_week = find_week_length()
                for l in range(1, len_week+1) :
                    click_week(l)
                    click_genre()
                    click_search()

                    # 크롤링 및 저장
                    crawling_and_save_to_csv(j, k, l)
        
        else :
            for k in range(1, len_month+1) :
                click_month(k)

                len_week = find_week_length()
                for l in range(1, len_week+1) :
                    click_week(l)
                    click_genre()
                    click_search()

                    # 크롤링 및 저장
                    crawling_and_save_to_csv(j, k, l)
            
        if i == stop_era and j == stop_year : break    
                            
    if i == stop_era : break;
        
driver.quit()

2021-01-1 2021-01-2 2021-01-3 2021-01-4 2021-02-1 2021-02-2 2021-02-3 2021-02-4 2021-03-1 2021-03-2 2021-03-3 2021-03-4 2021-04-1 2021-04-2 2021-04-3 2021-04-4 2021-04-5 2021-05-1 2021-05-2 2021-05-3 2021-05-4 2021-06-1 2021-06-2 2021-06-3 2020-01-1 2020-01-2 2020-01-3 2020-01-4 2020-01-5 2020-02-1 2020-02-2 2020-02-3 2020-02-4 2020-03-1 2020-03-2 2020-03-3 2020-03-4 2020-04-1 2020-04-2 2020-04-3 2020-04-4 2020-04-5 2020-05-1 2020-05-2 2020-05-3 2020-05-4 2020-06-1 2020-06-2 2020-06-3 2020-06-4 2020-07-1 2020-07-2 2020-07-3 2020-07-4 2020-07-5 2020-08-1 2020-08-2 2020-08-3 2020-08-4 2020-09-1 2020-09-2 2020-09-3 2020-09-4 2020-10-1 2020-10-2 2020-10-3 2020-10-4 2020-10-5 2020-11-1 2020-11-2 2020-11-3 2020-11-4 2020-12-1 2020-12-2 2020-12-3 2020-12-4 2020-12-5 2019-01-1 2019-01-2 2019-01-3 2019-01-4 2019-01-5 2019-02-1 2019-02-2 2019-02-3 2019-02-4 2019-03-1 2019-03-2 2019-03-3 2019-03-4 2019-04-1 2019-04-2 2019-04-3 2019-04-4 2019-05-1 2019-05-2 2019-05-3 2019-05-4 2019-05-5 2019-06-1 